# Introduction

This notebook shows an example of relaxing the LiFePO4 crystal.

In [10]:
import warnings

import numpy as np

from pymatgen.ext.matproj import MPRester
from pymatgen.util.coord import pbc_diff

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")

In [11]:
mpr = MPRester()
lfp = mpr.get_structure_by_material_id("mp-19017")  # This is LiFePO4.

lfp_strained = lfp.copy()  # We create a copy.
# Create a random strain between -5% and 5% for each direction
strains = np.random.uniform(low=-0.05, high=0.05, size=3)
lfp_strained.apply_strain(strains)
# In addition to the lattice strains, we also perturb the atoms by a distance of 0.1 angstrom.
lfp_strained.perturb(0.1)

In [12]:
relaxer = Relaxer()
%time relax_results = relaxer.relax(lfp_strained)
relaxed = relax_results["final_structure"]

CPU times: user 11.3 s, sys: 2.17 s, total: 13.5 s
Wall time: 7.72 s


Note that the relaxation only took < 20s.

In [13]:
print(f"Original lattice parameters are {[round(x, 3) for x in lfp.lattice.abc]}")
print(f"Strained lattice parameters are {[round(x, 3) for x in lfp_strained.lattice.abc]}")
print(f"Relaxed lattice parameters are {[round(x, 3) for x in relaxed.lattice.abc]}")

Original lattice parameters are [4.746, 10.444, 6.09]
Strained lattice parameters are [4.751, 10.912, 5.849]
Relaxed lattice parameters are [4.75, 10.444, 6.106]


In [5]:
print(f"Diff in fractional coords:\n{pbc_diff(lfp.frac_coords, relaxed.frac_coords)}")

Diff in fractional coords:
[[-0.00387997 -0.00279335 -0.00278419]
 [-0.0105322   0.00017964 -0.00350815]
 [ 0.00567224 -0.00023636 -0.00763473]
 [ 0.00193715  0.00778645 -0.0020107 ]
 [ 0.00225437  0.00170045  0.00172863]
 [ 0.00097444  0.00094502  0.00388633]
 [ 0.00345818  0.00368189  0.0024407 ]
 [-0.00038704  0.00417402  0.00208854]
 [ 0.00349925  0.00451124 -0.00329377]
 [ 0.00301799  0.00162616  0.00426465]
 [-0.00035351  0.0040381   0.00160278]
 [-0.00239293  0.00165153 -0.00127454]
 [ 0.00438634  0.00427503 -0.00071235]
 [ 0.00422623  0.002672    0.00449112]
 [-0.00079353  0.0043336  -0.00153869]
 [-0.0033543   0.00193773 -0.00280839]
 [-0.00151142  0.00393776 -0.00398973]
 [-0.00481124  0.00160034 -0.00323102]
 [ 0.00471888  0.0048146  -0.00417542]
 [ 0.0007874   0.00072707  0.00049835]
 [ 0.0041545   0.00410332 -0.00345285]
 [ 0.00412935  0.0025092   0.00449315]
 [ 0.00459609  0.0002639   0.00462091]
 [ 0.00272673  0.00434772 -0.00369026]
 [ 0.00018383  0.00028395  0.00456975

Quite clealy, the relaxation using the M3GNet universal IAP has brought the lattice parameters much closer to the original DFT one and the coordinates are also within $10^{-3}$ of the original fractional coordinates.